<a href="https://colab.research.google.com/github/secutron/ACTOR/blob/master/_test_clova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import hmac
import base64
import time
import requests
import json
import os
import sys
import urllib.request
import shutil

In [2]:
class ChatbotMessageSender:

    # chatbot api gateway url
    ep_path = 'https://3svep1xfda.apigw.ntruss.com/custom/v1/6887/558fe5674f162b84dd4bd83029e086ea1d96f9457826544f95b49924aa453fba'
    # chatbot custom secret key
    secret_key = 'VGJiQ2NTUkF3SWJUeUZ3QXlYdW9nZ0dUTVdOY2Vuc1c='

    def req_message_send(self, text):

        timestamp = self.get_timestamp()
        request_body = {
            'version': 'v2',
            'userId': 'U47b00b58c90f8e47428af8b7bddcda3d1111111',
            'timestamp': timestamp,
            'bubbles': [
                {
                    'type': 'text',
                    'data': {
                        'description': text
                    }
                }
            ],
            'event': 'send'
        }

        ## Request body
        encode_request_body = json.dumps(request_body).encode('UTF-8')

        ## make signature
        signature = self.make_signature(self.secret_key, encode_request_body)

        ## headers
        custom_headers = {
            'Content-Type': 'application/json;UTF-8',
            'X-NCP-CHATBOT_SIGNATURE': signature
        }

        # print("## Timestamp : ", timestamp)
        # print("## Signature : ", signature)
        # print("## headers ", custom_headers)
        # print("## Request Body : ", encode_request_body)

        ## POST Request
        response = requests.post(headers=custom_headers, url=self.ep_path, data=encode_request_body)

        return response

    @staticmethod
    def get_timestamp():
        timestamp = int(time.time() * 1000)
        return timestamp

    @staticmethod
    def make_signature(secret_key, request_body):

        secret_key_bytes = bytes(secret_key, 'UTF-8')

        signing_key = base64.b64encode(hmac.new(secret_key_bytes, request_body, digestmod=hashlib.sha256).digest())

        return signing_key

In [3]:
#챗봇 함수
def APIchat(text):
  if __name__ == '__main__':

      res = ChatbotMessageSender().req_message_send(text)
      if(res.status_code == 200):
          
          return(res.json()['bubbles'][0]['data']['description'])
          # print(res.read().decode("UTF-8"))

TTS(음성 답변용)

In [4]:
def answer_voice(text):
  client_id = "zyqwh8x83v"
  client_secret = "gE21EZDEXp8esM8K2JYUMFXllkanun0Jd6UA7OXG"
  encText = urllib.parse.quote(text)
  data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
  url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
  request = urllib.request.Request(url)
  request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
  request.add_header("X-NCP-APIGW-API-KEY",client_secret)
  response = urllib.request.urlopen(request, data=data.encode('utf-8'))
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      with open('answer.mp3', 'wb') as f:
          f.write(response_body)
      source = 'answer.mp3'
      #출력될 음성은 로컬에서 저장된 후 자바스크립트를 통해 재생됨
      destination = '/content/sounds/answer.mp3'
      shutil.move(source, destination)
  else:
    errormsg = ("Error Code:" + rescode)
    return errormsg

STT(음성 입력용)

In [5]:
# 현 코드에서는 미사용
def stt():
  import sys
  import requests
  client_id = "zyqwh8x83v"
  client_secret = "gE21EZDEXp8esM8K2JYUMFXllkanun0Jd6UA7OXG"
  lang = "Kor" # 언어 코드 ( Kor, Jpn, Eng, Chn )
  url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=" + lang
  data = open('/content/sounds/answer.mp3', 'rb')
  headers = {
      "X-NCP-APIGW-API-KEY-ID": client_id,
      "X-NCP-APIGW-API-KEY": client_secret,
      "Content-Type": "application/octet-stream"
  }
  response = requests.post(url,  data=data, headers=headers)
  rescode = response.status_code
  if(rescode == 200):
      restext = (eval(response.text))['text']
      return restext
  else:
      return("Error : " + response.text)

In [9]:
#네이버 클라우드플랫폼 챗봇 함수(str 형태로 메시지를 입력하면 동작)
reply = APIchat("농심을 대표하는 라면의 이름?")
print(reply)
#answer_voice(reply)

None
